In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    shuffle=True,
    image_size=(256,256),
    batch_size=32
)

In [ ]:
class_names=dataset.class_names

In [ ]:
len(dataset)

In [ ]:
for image_batch, label_batch in dataset.take(1):
    plt.imshow(image_batch[0].numpy().astype('uint8'))
    plt.title(class_names[label_batch[0]])
    print(image_batch[0].numpy())
    print(label_batch.numpy())

In [ ]:
epochs=50
training=0.8
validation=0.1
test=0.1

In [ ]:
train_ds=dataset.take(54)
test_ds=dataset.skip(54)

In [ ]:
validation_ds= test_ds.take(6)
test_ds=test_ds.skip(6)

In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE) 
val_ds=validation_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE) 
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE) 

In [ ]:
resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1./255)
])

In [ ]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
model= models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3), activation='relu', input_shape=(32,256,256,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
    
])
model.build(input_shape=(32,256,256,3))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'
                       ])

In [ ]:
model.fit(
    train_ds,
    epochs=epochs,
    batch_size=32,
    verbose=1,
    validation_data=validation_ds
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
import numpy as np
for image_batch, labels_batch in test_ds.take(1):
    
    first_image=image_batch[0].numpy().astype('uint8')
    first_label= labels_batch[0].numpy()
    
    print('First image to predict')
    plt.imshow(first_image)
    print('Actual label:', class_names[first_label])
    
    batch_prediction=model.predict(image_batch)
    print('Predicted label:',class_names[np.argmax(batch_prediction[0])])

    

In [ ]:
def predict(model, img):
    
    img_array= tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array= tf.expand_dims(img_array,0)
    
    predictions=model.predict(img_array)
    
    predicted_class= class_names[np.argmax(predictions[0])]
    confidence= round(100* (np.max(predictions[0])),2)
    
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(12,12))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        
        predicted_class, confidence= predict(model, images[i].numpy())
        actual_class= class_names[labels[i]]
        
        plt.title(f"Actual: {actual_class},/n Predicted: {predicted_class}. \n Confidence: {confidence}")
        
        plt.axis('off')
        
        

In [ ]:
model_version=1
model.save(f'./models/')